In [ ]:
import re
import os

FOLDER = "cases"

import json
import pandas as pd
from dataclasses import dataclass

def subtract_df(df_original, df_subset):
    # Слияние двух DataFrame с помощью внешнего слияния (outer join) и индикатора
    merged_df = pd.merge(df_original, df_subset, how='outer', indicator=True)

    # Выбираем только те строки, которые присутствуют в оригинальном DataFrame
    df_result = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

    # Если вы хотите сохранить только столбцы оригинального DataFrame
    df_result = df_result[df_original.columns]

    return df_result



def parse_conditions_to_code(conditions):
    python_code = conditions.replace("AND", "&").replace("OR", "|").replace("NOT", "~")

    python_code = python_code.replace('"Площадка"', 'df_orig["Площадка"]')
    python_code = python_code.replace('"Тип используемых прав" ', 'df_orig["Тип используемых прав"]')
    python_code = python_code.replace('"Территория"', 'df_orig["Территория"]')
    python_code = python_code.replace('"Тип контента"', 'df_orig["Тип контента"]')
    python_code = python_code.replace('"Вид использования  контента"', 'df_orig["Вид использования  контента"]')
    python_code = python_code.replace('"Исполнитель"', 'df_orig["Исполнитель"]')
    python_code = python_code.replace('"Название трека"', 'df_orig["Название трека"]')
    python_code = python_code.replace('"Название альбома"', 'df_orig["Название альбома"]')
    python_code = python_code.replace('"Автор слов"', 'df_orig["Автор слов"]')
    python_code = python_code.replace('"Автор музыки"', 'df_orig["Автор музыки"]')
    python_code = python_code.replace('"ISRC контента"', 'df_orig["ISRC контента"]')
    python_code = python_code.replace('"UPC альбома"', 'df_orig["UPC альбома"]')
    python_code = python_code.replace('"Копирайт"', 'df_orig["Копирайт"]')

    python_code = re.sub(r'\s+', ' ', python_code).strip()

    python_code = f'df_orig[{python_code}]'

    return python_code


def calc_sum(df):
    res = 0
    for index, row in df.iterrows():
        res += (row.iloc[c["SUM_1"]] + row.iloc[c["SUM_2"]])
    return round(res, 2)


@dataclass
class Data:
    license: int
    track: int
    name: str

def parse_filename(name: str) -> Data:
    name = name.replace(".txt", "")
    d = name.split(",")

    license = int(d[0].replace("license=", ""))
    track = int(d[1].replace("track=", ""))
    name = d[2].replace("name=", "")

    return Data(license=license, track=track, name=name)

with open("config.json", "r", encoding="utf-8") as f:
    c = json.loads(f.read())

df_orig = pd.read_excel(c["filename"], header=c["header_index"])

col_len = len(df_orig.columns)
if  col_len > c["col_len"]:
    df_orig = df_orig.iloc[:, :c["col_len"]-col_len]


all_case_dfs = list()
case_dfs = dict()

for case in os.listdir(FOLDER):
    if ".DS_Store" in case:
        continue
    case_path = os.path.join(FOLDER, case)
    res = 0
    
    case_dfs[case] = list()
    
    if os.path.isfile(case_path):
        with open(case_path, "r", encoding="utf-8") as f:
            conditions = f.read()
        code_output = parse_conditions_to_code(conditions)
        data = parse_filename(case)
        df = eval(code_output)

        all_case_dfs.append(df)
        case_dfs[case].append(df)

        res += calc_sum(df) * (data.per/100)
        print(round(res, 2), data.name)
    else:
        for subcase in os.listdir(case_path):
            r = 0
            with open(os.path.join(case_path, subcase), "r", encoding="utf-8") as f:
                conditions = f.read()
            code_output = parse_conditions_to_code(conditions)
            df = eval(code_output)

            all_case_dfs.append(df)
            case_dfs[case].append(df)

            data = parse_filename(subcase)
            r = calc_sum(df) * (data.per/100)

            res += r
        data.name = case

        print(round(res, 2), data.name)



In [ ]:
import os

def rename_files_recursively(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            old_file_path = os.path.join(root, file)
            new_file_path = old_file_path.replace("licence=", "license=")
            os.rename(old_file_path, new_file_path)
            print(f"Renamed '{old_file_path}' to '{new_file_path}'")

# Example usage:
folder_path = 'cases'  # Change this to your folder path

rename_files_recursively(folder_path)


In [ ]:
with open("config.json", "r", encoding="utf-8") as f:
    c = json.loads(f.read())

df_orig = pd.read_excel(c["filename"], header=c["header_index"])

col_len = len(df_orig.columns)
if  col_len > c["col_len"]:
    df_orig = df_orig.iloc[:, :c["col_len"]-col_len]

In [ ]:
print(df_orig[(df_orig["Исполнитель"].isna()) & (df_orig["Название трека"] == "Грустные песни")])

In [2]:
import os

def rename_files_recursively(start_dir):
    for dirpath, _, filenames in os.walk(start_dir):
        for filename in filenames:
            if filename.endswith('.txt') and ',' in filename:
                # Extract the base name without the extension
                base_name = filename[:-4]
                parts = base_name.split(',')
                # Find the part that starts with "name="
                name_part = next((part for part in parts if part.startswith('name=')), None)
                if name_part:
                    # Remove the name part from the list
                    parts.remove(name_part)
                    # Construct the new filename with the name part at the beginning
                    new_base_name = f"{name_part},{','.join(parts)}"
                    new_filename = f"{new_base_name}.txt"
                    old_file_path = os.path.join(dirpath, filename)
                    new_file_path = os.path.join(dirpath, new_filename)
                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    print(f"Renamed '{old_file_path}' to '{new_file_path}'")

# Replace 'your_directory' with the path to the directory containing your files
rename_files_recursively('cases')


Renamed 'cases\license=100,track=35,name=Вика Радисева (гокки).txt' to 'cases\name=Вика Радисева (гокки),license=100,track=35.txt'
Renamed 'cases\license=100,track=35,name=Вика Радисева (друнк).txt' to 'cases\name=Вика Радисева (друнк),license=100,track=35.txt'
Renamed 'cases\license=100,track=70,name=Апрелия.txt' to 'cases\name=Апрелия,license=100,track=70.txt'
Renamed 'cases\license=100,track=70,name=Милохин и остальные.txt' to 'cases\name=Милохин и остальные,license=100,track=70.txt'
Renamed 'cases\license=100,track=70,name=Панчер Копирайт.txt' to 'cases\name=Панчер Копирайт,license=100,track=70.txt'
Renamed 'cases\Алтунин\license=100,track=60,name=1.txt' to 'cases\Алтунин\name=1,license=100,track=60.txt'
Renamed 'cases\Алтунин\license=50,track=60,name=2.txt' to 'cases\Алтунин\name=2,license=50,track=60.txt'
Renamed 'cases\Алтунин\license=70,track=60,name=3.txt' to 'cases\Алтунин\name=3,license=70,track=60.txt'
Renamed 'cases\Вика Радисева\license=100,track=50,name=2.txt' to 'cases\